In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [5]:
mnist, info = tfds.load('mnist', as_supervised=True, with_info=True)
print(info)

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.0,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)



In [6]:
import numpy as np
train_data, test_data = mnist['train'], mnist['test']
train_images = []
train_labels = []
test_images = []
test_labels = []
for i, l in train_data:
  train_images.append(i.numpy())
  train_labels.append(l.numpy())
for i, l in test_data:
  test_images.append(i.numpy())
  test_labels.append(l.numpy())

In [7]:
train_images = np.array(train_images)
#np.expand_dims(train_images, axis=3)
test_images = np.array(test_images)
#np.expand_dims(test_images, axis=3)

In [11]:
print(train_images[0])

[[[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]


In [8]:
train_labels_real = []
for i in train_labels:
  lbl = np.zeros(10)
  lbl[i] = 1
  train_labels_real.append(lbl)
test_labels_real = []
for i in test_labels:
  lbl = np.zeros(10)
  lbl[i] = 1
  test_labels_real.append(lbl)



In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1.0/255.)

validation_gen = ImageDataGenerator(rescale=1.0/255.)

In [3]:
model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(16, (3,3),  activation='relu', input_shape=(28, 28, 1)),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(1024, activation='relu'),
                             tf.keras.layers.Dense(10, activation='softmax')                          
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 2704)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2769920   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total params: 2,780,330
Trainable params: 2,780,330
Non-trainable params: 0
_________________________________________________________________


In [11]:
from tensorflow.keras.optimizers import  RMSprop
model.compile(optimizer=RMSprop(lr=0.0001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [12]:
model.fit(train_gen.flow(train_images, train_labels_real, batch_size=100),
          validation_data = train_gen.flow(test_images, test_labels_real, batch_size=100),
          steps_per_epoch=600,
          verbose=1,
          epochs=20)

Epoch 1/20
600/600 [==============================] - 4s 6ms/step - loss: 0.4290 - accuracy: 0.8997 - val_loss: 0.1816 - val_accuracy: 0.9486
Epoch 2/20
600/600 [==============================] - 3s 6ms/step - loss: 0.1583 - accuracy: 0.9548 - val_loss: 0.1217 - val_accuracy: 0.9655
Epoch 3/20
600/600 [==============================] - 3s 5ms/step - loss: 0.1099 - accuracy: 0.9691 - val_loss: 0.0948 - val_accuracy: 0.9710
Epoch 4/20
600/600 [==============================] - 3s 6ms/step - loss: 0.0846 - accuracy: 0.9767 - val_loss: 0.0773 - val_accuracy: 0.9756
Epoch 5/20
600/600 [==============================] - 3s 6ms/step - loss: 0.0690 - accuracy: 0.9809 - val_loss: 0.0637 - val_accuracy: 0.9798
Epoch 6/20
600/600 [==============================] - 3s 6ms/step - loss: 0.0582 - accuracy: 0.9834 - val_loss: 0.0604 - val_accuracy: 0.9800
Epoch 7/20
600/600 [==============================] - 3s 5ms/step - loss: 0.0505 - accuracy: 0.9857 - val_loss: 0.0531 - val_accuracy: 0.9822
Epoch 

In [13]:
import tensorflowjs as tfjs

ModuleNotFoundError: ignored

In [ ]:
!pip install tensorflowjs

In [16]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "/tmp/weights")

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [17]:
!zip -r /tmp/file.zip /tmp/weights/


  adding: tmp/weights/ (stored 0%)
  adding: tmp/weights/group1-shard3of3.bin (deflated 7%)
  adding: tmp/weights/group1-shard1of3.bin (deflated 7%)
  adding: tmp/weights/model.json (deflated 74%)
  adding: tmp/weights/group1-shard2of3.bin (deflated 7%)


In [18]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('/tmp/file.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>